In [171]:
# Packages
import sys; sys.path.append('..')
import src.seed as seed
from src.data import data_load


In [172]:
# Parameters
path = '../data/example.txt'  # dataset file path
init_rate = 0.05  # initial influence rate, recommend from 0 to 0.05(will have to increase for smaller graphs)
threshold = 0.5  # threshold of influence, recommend from 0 to 1

# Seed selection policy, option: 'degree', 'random',
# 'degree_discount', 'degree_neighbor', 'degree_neighbor_fix', 'mia'
policy = 'degree'  

In [173]:
# Seed selection
nodes, edges = data_load()
seeds_number = int(len(nodes) * init_rate)
if policy == 'degree':
    seeds = seed.degree(edges, seeds_number)
elif policy == 'random':
    seeds = seed.random(nodes, seeds_number)
elif policy == 'degree_discount':
    seeds = seed.degree_discount(edges, seeds_number)
elif policy == 'degree_neighbor':
    seeds = seed.degree_neighbor(edges, seeds_number)
elif policy == 'degree_neighbor_fix':
    seeds = seed.degree_neighbor_fix(edges, seeds_number)
elif policy == 'mia':
    seeds = seed.mia(nodes, edges, seeds_number)
else: 
    raise NameError("Unknown policy")
print(f'Number of Seeds: {len(seeds)}')

Number of Nodes: 34
Number of Edges: 156
Number of Seeds: 1


In [174]:
print(seeds)

[33]


In [175]:
import numpy as np


def influence_count(nodes, edges, seeds, threshold):
    ''' Calculate influent result
    Args:
        nodes (list) [#node]: nodes list of the graph;
        edges (list of list) [#edge, 2]: edges list of the graph;
        seeds (list) [#seed]: selected seeds;
        threshold (float): influent threshold, between 0 and 1;
    Return:
        final_actived_node (list): list of influent nodes;
    '''
    in_degree = {}
    inactive_nodes = []
    active_nodes = []
    nodes_status = {}

    for edge in edges: 
        if edge[0] in seeds:
            active_nodes.append(edge[0])
        else:
            inactive_nodes.append(edge[0])
        if edge[1] in seeds:
            active_nodes.append(edge[1])
        else:
            inactive_nodes.append(edge[1])
        if edge[1] in in_degree:
            in_degree[edge[1]] += 1
        else:
            in_degree[edge[1]] = 1

    active_nodes = list(set(active_nodes))
    inactive_nodes = list(set(inactive_nodes))

    for node in nodes:
        nodes_status[node] = 0
    for node in active_nodes:
        nodes_status[node] = 1
            
    while(active_nodes):
        new_actived_nodes = []
        for edge in edges:
            if nodes_status[edge[0]] == 1:
                if nodes_status[edge[1]] == 0:
                    p = np.array([1 - threshold / in_degree[edge[1]], threshold / in_degree[edge[1]]])
                    flag = np.random.choice([0, 1], p=p.ravel())
                    if flag:
                        new_actived_nodes.append(edge[1])
        for node in active_nodes:
            nodes_status[node] = 2
        for node in new_actived_nodes:
            nodes_status[node] = 1
        active_nodes = new_actived_nodes

    final_actived_node = 0
    a=[]
    for node in nodes:
        if nodes_status[node] == 2:
            final_actived_node += 1
            a.append(node)
    return a

In [176]:
# Calculate Influence
b= influence_count(nodes, edges, seeds, threshold)
print(b)

[9, 18, 26, 33]


In [177]:
a=[]
influential_node=[]
n=200
for i in range(0,n):
    a=a+(influence_count(nodes, edges, seeds, threshold))
for i in range(0,34):
    if a.count(i)>=n/4 :
        influential_node.append(i)
print(influential_node)

[9, 15, 18, 22, 26, 33]
